In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from tools import data_prep

In [2]:
X, y = data_prep()

D:\Programs\conda\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41432 entries, 0 to 41475
Data columns (total 31 columns):
PurchaseID                           41432 non-null int64
PurchaseTimestamp                    41432 non-null int64
PurchaseDate                         41432 non-null object
Auction                              41432 non-null object
VehYear                              41432 non-null float64
Make                                 41432 non-null object
Color                                41432 non-null object
Transmission                         41432 non-null object
WheelTypeID                          41432 non-null object
WheelType                            41380 non-null object
VehOdo                               41432 non-null float64
Nationality                          41432 non-null object
Size                                 41432 non-null object
TopThreeAmericanName                 41432 non-null object
MMRAcquisitionAuctionAveragePrice    41416 non-null object
MMRAcq

In [3]:
#from sklearn.preprocessing import LabelEncoder
#labelEncoder = LabelEncoder()

#categorical_feature_mask = df_final.dtypes==object
#categorical_cols = df_final.columns[categorical_feature_mask].tolist()
#df_final[categorical_cols] = df_final[categorical_cols].apply(lambda col: labelEncoder.fit_transform(col))
#df_final.values[:,7] = labelEncoder.fit_transform(df_final.values[:,7])

#df_final

# set the random seed - consistent
rs = 10

# train test split
X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.3, stratify=y, random_state=rs)

D:\Programs\conda\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


In [24]:
# initialise a standard scaler object
scaler = StandardScaler()

# visualise min, max, mean and standard dev of data before scaling
print("Before scaling\n-------------")
for i in range(5):
    col = X_train[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

# learn the mean and std.dev of variables from training data
# then use the learned values to transform training data
X_train = scaler.fit_transform(X_train, y_train)

print("After scaling\n-------------")
for i in range(5):
    col = X_train[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

# use the statistic that you learned from training to transform test data
# NEVER learn from test data, this is supposed to be a set of dataset
# that the model has never seen before
X_test = scaler.transform(X_test)

Before scaling
-------------
Variable #0: min 0.0, max 1.0, mean 0.04 and std dev 0.19
Variable #1: min 577.0, max 480444.0, mean 72384.47 and std dev 14998.09
Variable #2: min 0.0, max 33543.0, mean 5946.68 and std dev 2568.24
Variable #3: min 0.0, max 36701.0, mean 7178.38 and std dev 2844.14
Variable #4: min 0.0, max 36726.0, mean 8228.66 and std dev 3252.30
After scaling
-------------
Variable #0: min -0.1949472296104162, max 5.129593285312624, mean 0.00 and std dev 1.00
Variable #1: min -4.7877757075676, max 27.207441862755417, mean 0.00 and std dev 1.00
Variable #2: min -2.315464923295065, max 10.745211522747928, mean -0.00 and std dev 1.00
Variable #3: min -2.52391795848795, max 10.38015377969556, mean 0.00 and std dev 1.00
Variable #4: min -2.530107339282775, max 8.762218203273306, mean -0.00 and std dev 1.00


In [4]:
model = LogisticRegression(random_state=rs)

# fit it to training data
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
# training and test accuracy
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

# classification report on test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


Train accuracy: 0.825455997869791
Test accuracy: 0.8232370301335818
             precision    recall  f1-score   support

          0       0.77      0.92      0.84      1609
          1       0.90      0.73      0.80      1610

avg / total       0.84      0.82      0.82      3219



In [27]:
feature_names = X.columns
coef = model.coef_[0]

# limit to 20 features, you can comment the following line to print out everything
coef = coef[:20]

for i in range(len(coef)):
    print(feature_names[i], ':', coef[i])

Transmission : -0.06822026349210418
VehOdo : 0.15152118438613474
MMRAcquisitionAuctionAveragePrice : -1.3802951674635233
MMRAcquisitionAuctionCleanPrice : 0.5924495249311702
MMRAcquisitionRetailAveragePrice : 1.2258851395157986
MMRAcquisitonRetailCleanPrice : -0.05321443060115353
MMRCurrentAuctionAveragePrice : 0.68008763880614
MMRCurrentAuctionCleanPrice : -0.3412170106727569
MMRCurrentRetailAveragePrice : -0.0921957265534876
MMRCurrentRetailCleanPrice : -0.41957346998343953
MMRCurrentRetailRatio : -0.06541074269927825
VehBCost : -0.37490759532762263
WarrantyCost : 0.08317583122008965
Auction_ADESA : 2.06569972872485
Auction_MANHEIM : -0.8391635647189372
Auction_OTHER : -0.9037717245362028
VehYear_2001.0 : 0.22098137368059795
VehYear_2002.0 : 0.26684034588858735
VehYear_2003.0 : 0.2373170329925653
VehYear_2004.0 : 0.17166976022239985


In [28]:
# grab feature importances from the model and feature name from the original X
coef = model.coef_[0]
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(np.absolute(coef))
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', coef[i])

Auction_ADESA : 2.06569972872485
VNST_AZ : 1.8048255117114467
MMRAcquisitionAuctionAveragePrice : -1.3802951674635233
VNST_OK : 1.270164390498947
MMRAcquisitionRetailAveragePrice : 1.2258851395157986
Auction_OTHER : -0.9037717245362028
Auction_MANHEIM : -0.8391635647189372
VNST_CO : -0.8096213013663035
MMRCurrentAuctionAveragePrice : 0.68008763880614
VNST_PA : 0.669406061904637
VehYear_2008.0 : -0.6623714131764916
MMRAcquisitionAuctionCleanPrice : 0.5924495249311702
VNST_MO : 0.570462431964441
VNST_TX : -0.5348395061384371
VNST_FL : -0.48501297235417323
VNST_GA : -0.4633650440807044
VNST_VA : -0.4547188987170395
VNST_MS : 0.4284652650741305
MMRCurrentRetailCleanPrice : -0.41957346998343953
VNST_NJ : 0.37943657146570997


In [29]:
# grid search CV
params = {'C': [pow(10, x) for x in range(-6, 4)]}

# use all cores to tune logistic regression with C parameter
cv = GridSearchCV(param_grid=params, estimator=LogisticRegression(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

# test the best model
print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.825455997869791
Test accuracy: 0.8232370301335818
             precision    recall  f1-score   support

          0       0.77      0.92      0.84      1609
          1       0.90      0.73      0.80      1610

avg / total       0.84      0.82      0.82      3219

{'C': 1}
